## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [2]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([1, 0, 0,  ..., 1, 1, 0], dtype=torch.uint8)

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [3]:
# try project here!
def gen_parallel_db(data_base, index):
    return torch.cat((data_base[0:index], data_base[index+1:]))

In [4]:
def gen_all_parallel_db(data_base):
    parallel_dbs = []
    for i in range(len(data_base)):
        parallel_dbs.append(gen_parallel_db(data_base, i))
    return parallel_dbs

In [5]:
def gen_db_and_parallels(num_elements):
    db = torch.rand(num_elements) > 0.5
    return db, gen_all_parallel_db(db)

In [6]:
gen_db_and_parallels(4)

(tensor([0, 1, 0, 1], dtype=torch.uint8),
 [tensor([1, 0, 1], dtype=torch.uint8),
  tensor([0, 0, 1], dtype=torch.uint8),
  tensor([0, 1, 1], dtype=torch.uint8),
  tensor([0, 1, 0], dtype=torch.uint8)])

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating whether the output of a query changes when we remove someone from the database. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [7]:
db, pdbs = gen_db_and_parallels(5000)

In [8]:
def query(db):
    return db.sum()

In [9]:
full_db_result = query(db)

In [10]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [11]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). This value is called "sensitivity", and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [12]:
# try this project here!
def sensitivity(query, n_entries):
    # Initialize the database
    database, parallel_databases = gen_db_and_parallels(n_entries)
    
    # Run query over all the databases
    full_db_result = query(database)
    sensitivity = 0
    
    for db in parallel_databases:
        pdb_result = query(db)
        distance = torch.abs(pdb_result-full_db_result)
    
        if distance > sensitivity:
            sensitivity = distance
    return sensitivity
        

In [13]:
# Define query
def query(db):
    return db.float().mean()

In [14]:
sensitivity(query, 1000)

tensor(0.0005)

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [15]:
# try this project here!
def threshold(database, threshold_val = 5):
    return (database.sum() > threshold_val).float()

In [16]:
sensitivity_arr = [10 for i in range(10)]
sensitivity_arr

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [17]:
for index in range(len(sensitivity_arr)):
    sensitivity_arr[index] = sensitivity(threshold, sensitivity_arr[index])
sensitivity_arr

[0, 0, 0, 0, 0, tensor(1.), 0, 0, 0, 0]

# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [18]:
# Create database with at least ten rows
database = torch.rand(20) > 0.5
database

tensor([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1],
       dtype=torch.uint8)

In [19]:
total_sum = database.sum()
total_sum

tensor(9)

In [20]:
partial_sum = torch.cat((database[0:10], database[11:])).sum()
partial_sum

tensor(8)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, the this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [21]:
# try this project here!
# Approach 1
import random

def local_diff_priv(query, len_db):
    # Generate the database
    database = torch.rand(len_db)>0.5
    
    # Get the result of the query on the initial database
    original_result = query(database)
    
    # Randomise the database to allow for plausible denyability
    for index, entry in enumerate(database):
        # Generate the random coin flips
        coin_flip1, coin_flip2 = random.randint(0, 1), random.randint(0, 1)
        
        # Check if the first flip is a not head
        if coin_flip1 == 1:
            continue
        else:
            database[index] = coin_flip2
            
    # Now generate a new randomised result.
    randomised_result = query(database) *2 - 0.5
    
    # Return the original result and the randomised result
    return original_result, randomised_result  

# Define query function
def query(data_base):
    return data_base.float().mean()

# Test output for different database sizes
original_res, randomised_res = local_diff_priv(query, 1000)
print("Original Result: ", original_res)
print("Randomised Result: ", randomised_res)

Original Result:  tensor(0.5220)
Randomised Result:  tensor(0.5300)


In [22]:
# Approach 2
database, paralle_database = gen_db_and_parallels(100)
database

tensor([1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 1], dtype=torch.uint8)

In [23]:
true_result = database.float().mean()
true_result

tensor(0.5100)

In [24]:
first_coin_flip = (torch.rand(len(database))>0.5).float()
second_coin_flip = (torch.rand(len(database))>0.5).float()

In [25]:
database.float() * first_coin_flip

tensor([1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1.])

In [26]:
(1- first_coin_flip)*second_coin_flip

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
        0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 0., 0.])

In [27]:
augmented_database = (database.float() * first_coin_flip) + ((1-first_coin_flip)*second_coin_flip)

In [28]:
augmented_database

tensor([1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
        1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.,
        1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
        1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 1., 1., 1., 0., 1.])

In [29]:
# Functional approach 2
def query(db):
    # Calculate the true result
    true_result = torch.mean(db.float())
    
    # Flip the coins and cast to float
    first_coin_flip = (torch.rand(len(db))>0.5).float()
    second_coin_flip = (torch.rand(len(db))>0.5).float()
    
    # Calculate the result of the augmented database
    augmented_database = (db.float()*first_coin_flip) + ((1-first_coin_flip)*second_coin_flip)
    
    # Calculate the augmented result
    augmented_result = augmented_database.mean()
    
    # remove the skew of the augmented database
    unskewed_augmented_result = (augmented_result * 2) - 0.5
    
    return true_result, unskewed_augmented_result

In [30]:
db, pdbs = gen_db_and_parallels(1000)
print(query(db))

(tensor(0.5300), tensor(0.5540))


# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [31]:
# Approach 1
import random

def local_diff_priv(query, len_db, varying_noise):
    # Generate the database
    database = torch.rand(len_db)>0.5
    
    # Get the result of the query on the initial database
    original_result = query(database)
    
    # Randomise the database to allow for plausible denyability
    for index, entry in enumerate(database):
        # Generate the random coin flips
        coin_flip1, coin_flip2 = random.randint(0, 1), random.randint(0, 1)
        
        # Check if the first flip is a not head
        if coin_flip1 == 1:
            continue
        else:
            random_number = random.random()
            if random_number < varying_noise:
                database[index] = coin_flip2
            
    # Now generate a new randomised result.
    randomised_result = query(database) *2 - 0.5
    
    # Return the original result and the randomised result
    return original_result, randomised_result  

# Define query function
def query(data_base):
    return data_base.float().mean()

# Test output for different database sizes
original_res, randomised_res = local_diff_priv(query, 1000, 0.3)
print("Original Result: ", original_res)
print("Randomised Result: ", randomised_res)

Original Result:  tensor(0.5150)
Randomised Result:  tensor(0.5160)


# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [32]:
db, pdbs = gen_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(43.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [33]:
epsilon = 0.5

In [34]:
import numpy as np

In [35]:
database, parallel_database = gen_db_and_parallels(100)

In [36]:
def sum_query(db):
    return db.sum()

In [37]:
def laplacian_mechanism(database, query, sensitivity):
    beta = sensitivity/epsilon
    
    # Create a noise tensor.
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    
    return query(database) + noise
    

In [38]:
laplacian_mechanism(database, sum_query, 1)

tensor([61.9742], dtype=torch.float64)

# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [22]:
import numpy as np

In [23]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [24]:
preds2 = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions
preds2.shape

(10000, 10)

In [15]:
an_image = preds2[0]
label_counts = np.bincount(an_image, minlength = num_labels)
label_counts

array([0, 1, 1, 1, 2, 1, 0, 2, 1, 1], dtype=int64)

In [43]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [44]:
new_labels

[0,
 8,
 6,
 3,
 3,
 6,
 4,
 0,
 8,
 1,
 3,
 4,
 6,
 5,
 7,
 3,
 5,
 6,
 7,
 9,
 7,
 1,
 7,
 3,
 0,
 1,
 7,
 5,
 2,
 7,
 5,
 2,
 0,
 1,
 2,
 5,
 6,
 7,
 6,
 5,
 8,
 1,
 0,
 7,
 9,
 3,
 2,
 8,
 3,
 3,
 2,
 9,
 5,
 0,
 0,
 7,
 7,
 7,
 8,
 9,
 6,
 8,
 4,
 6,
 8,
 1,
 1,
 0,
 3,
 2,
 1,
 4,
 1,
 6,
 1,
 5,
 9,
 0,
 9,
 9,
 0,
 1,
 0,
 5,
 6,
 8,
 5,
 0,
 6,
 8,
 4,
 4,
 8,
 2,
 2,
 7,
 4,
 1,
 4,
 0,
 3,
 2,
 3,
 5,
 7,
 3,
 7,
 9,
 5,
 6,
 4,
 3,
 7,
 7,
 7,
 2,
 4,
 8,
 1,
 1,
 6,
 4,
 4,
 5,
 9,
 4,
 7,
 2,
 2,
 5,
 8,
 3,
 0,
 1,
 9,
 8,
 5,
 1,
 7,
 2,
 8,
 3,
 0,
 7,
 0,
 1,
 7,
 1,
 2,
 7,
 4,
 5,
 3,
 8,
 8,
 3,
 9,
 7,
 6,
 0,
 4,
 1,
 1,
 9,
 1,
 9,
 3,
 9,
 5,
 3,
 5,
 5,
 8,
 1,
 1,
 8,
 2,
 1,
 4,
 2,
 4,
 9,
 4,
 1,
 2,
 2,
 1,
 6,
 6,
 3,
 5,
 1,
 5,
 3,
 6,
 6,
 9,
 3,
 8,
 7,
 6,
 6,
 7,
 3,
 8,
 0,
 5,
 9,
 2,
 7,
 1,
 0,
 9,
 8,
 4,
 2,
 9,
 5,
 8,
 3,
 2,
 2,
 8,
 1,
 9,
 6,
 7,
 4,
 3,
 1,
 0,
 9,
 2,
 1,
 8,
 3,
 6,
 4,
 9,
 7,
 0,
 4,
 3,
 3,
 9,
 4,
 5,
 3,
 7,
 0,


# PATE Analysis

In [45]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [9]:
from syft.frameworks.torch.differential_privacy import pate

W0718 16:15:58.432812  7392 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0718 16:15:59.303534  7392 deprecation_wrapper.py:119] From C:\Users\Desh\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [10]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

preds[:,0:10] *= 0

data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments = 20)

assert data_dep_eps < data_ind_eps



NameError: name 'np' is not defined

In [48]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments = 20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


In [49]:
preds[:,0:50] *= 0

In [50]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

In [1]:
# import the neccesary modules and datasets.
import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader, TensorDataset
from torch import nn, optim, Tensor
import torch.nn.functional as F

# Define transforms for the images
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, ), (0.5, ))])
# Download the training set.
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [2]:
# Function to create the private records out of the large training set.
'''
Note: This function does not account for splitting of numbers that are not well rounded.
'''
def create_private_records(trainset, num_hospitals=100):
    # Number of records per hospital
    batch_size = len(trainset)//num_hospitals
    
    # Array to store the private records
    private_records = []
    
    # Loop through the train set labels set and split them accordingly.
    for starting_point in range(num_hospitals):
        start, end = starting_point*batch_size, (1+starting_point)*batch_size
        new_private_trainset = Subset(trainset, list(range(start, end)))
        loader = DataLoader(new_private_trainset, shuffle=True, batch_size=32, num_workers=2)
        private_records.append(loader)
        
    return private_records

In [3]:
# Define linear model that will be used in training our model
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # Add the dropout layers
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        # Make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [4]:
# Define function to train models
def train_models(trainloader):
    model = Classifier()
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.003)
    
    # Define the training epochs for each private hospital
    epochs = 15
    
    # Training loop
    for e in range(epochs):
        running_loss = 0
        for images, labels in trainloader:
            # Reset optimizer
            optimizer.zero_grad()
            # Get the log probabilities
            log_ps = model(images)
            # Calculate the loss
            loss = criterion(log_ps, labels)
            # Adjust params
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        else:
            print(f"Training loss: {running_loss}")
    return model

In [5]:
# Obtain the records
private_records = create_private_records(trainset)

In [6]:
# Now we want to train all the independent models separately.
all_models = []
for index, records in enumerate(private_records):
    print("Now training model {} -----------------------------------------------------------------------".format(index+1))
    model = train_models(records)
    all_models.append(model)
    print("Done training model {} -----------------------------------------------------------------------".format(index+1))

Now training model 1 -----------------------------------------------------------------------
Training loss: 42.17988610267639
Training loss: 34.21132159233093
Training loss: 23.209388077259064
Training loss: 16.776777863502502
Training loss: 14.185272514820099
Training loss: 12.870769768953323
Training loss: 9.775338917970657
Training loss: 9.153791666030884
Training loss: 8.375118017196655
Training loss: 7.862393572926521
Training loss: 7.543607696890831
Training loss: 6.199253387749195
Training loss: 6.26245553791523
Training loss: 5.244092170149088
Training loss: 5.302641116082668
Done training model 1 -----------------------------------------------------------------------
Now training model 2 -----------------------------------------------------------------------
Training loss: 42.992600202560425
Training loss: 32.60062611103058
Training loss: 25.372326374053955
Training loss: 17.97344535589218
Training loss: 15.957702398300171
Training loss: 12.3157417178154
Training loss: 10.9428

Training loss: 43.47314095497131
Training loss: 35.96499013900757
Training loss: 25.11954092979431
Training loss: 19.81469303369522
Training loss: 15.38940703868866
Training loss: 12.358566999435425
Training loss: 11.923464864492416
Training loss: 10.41647058725357
Training loss: 10.22058019042015
Training loss: 7.101173132658005
Training loss: 7.828352227807045
Training loss: 8.394767984747887
Training loss: 5.531789317727089
Training loss: 4.807037644088268
Training loss: 5.429578214883804
Done training model 13 -----------------------------------------------------------------------
Now training model 14 -----------------------------------------------------------------------
Training loss: 42.34721517562866
Training loss: 30.945751190185547
Training loss: 21.87588757276535
Training loss: 16.833271145820618
Training loss: 13.267526030540466
Training loss: 11.532493025064468
Training loss: 9.400379359722137
Training loss: 9.223765552043915
Training loss: 8.551461935043335
Training loss

Training loss: 41.96326446533203
Training loss: 31.39942717552185
Training loss: 25.22864866256714
Training loss: 17.771403551101685
Training loss: 13.983424603939056
Training loss: 11.448255836963654
Training loss: 10.878866463899612
Training loss: 9.58955293893814
Training loss: 8.659426629543304
Training loss: 8.331720367074013
Training loss: 6.946824446320534
Training loss: 5.8607466742396355
Training loss: 4.801329605281353
Training loss: 4.68777722120285
Training loss: 4.784396231174469
Done training model 25 -----------------------------------------------------------------------
Now training model 26 -----------------------------------------------------------------------
Training loss: 42.198020935058594
Training loss: 31.462796688079834
Training loss: 21.981869280338287
Training loss: 15.961946427822113
Training loss: 12.33141240477562
Training loss: 10.175198674201965
Training loss: 8.315587535500526
Training loss: 7.917085483670235
Training loss: 7.470530033111572
Training lo

Training loss: 41.52820932865143
Training loss: 27.713504672050476
Training loss: 15.863426625728607
Training loss: 12.475820064544678
Training loss: 9.16406625509262
Training loss: 9.108732551336288
Training loss: 6.633412092924118
Training loss: 5.623853944242001
Training loss: 5.335557021200657
Training loss: 4.483968801796436
Training loss: 4.652711883187294
Training loss: 3.3487970754504204
Training loss: 2.855001777410507
Training loss: 3.2129935771226883
Training loss: 4.360282331705093
Done training model 37 -----------------------------------------------------------------------
Now training model 38 -----------------------------------------------------------------------
Training loss: 42.39312529563904
Training loss: 32.91149187088013
Training loss: 23.332348883152008
Training loss: 16.06388407945633
Training loss: 14.604869842529297
Training loss: 10.653138875961304
Training loss: 9.157327324151993
Training loss: 8.1749277561903
Training loss: 7.396922737360001
Training loss:

Training loss: 43.0577073097229
Training loss: 33.14916229248047
Training loss: 23.186340808868408
Training loss: 19.79735255241394
Training loss: 14.102240145206451
Training loss: 12.146720260381699
Training loss: 10.784702807664871
Training loss: 9.241142958402634
Training loss: 7.796168506145477
Training loss: 6.375989891588688
Training loss: 5.6320067793130875
Training loss: 5.907248184084892
Training loss: 5.7095667496323586
Training loss: 4.898910470306873
Training loss: 4.358427323400974
Done training model 49 -----------------------------------------------------------------------
Now training model 50 -----------------------------------------------------------------------
Training loss: 43.04293084144592
Training loss: 33.00586378574371
Training loss: 23.03818017244339
Training loss: 17.91887491941452
Training loss: 14.500753998756409
Training loss: 11.291504919528961
Training loss: 9.281187176704407
Training loss: 8.898742988705635
Training loss: 7.953001409769058
Training los

Training loss: 42.98238968849182
Training loss: 31.135746359825134
Training loss: 21.03242039680481
Training loss: 15.019737839698792
Training loss: 13.531317591667175
Training loss: 10.448466390371323
Training loss: 9.459343433380127
Training loss: 8.501888528466225
Training loss: 8.674078941345215
Training loss: 6.134967342019081
Training loss: 5.601334787905216
Training loss: 5.17518625408411
Training loss: 4.072526972740889
Training loss: 4.889998555183411
Training loss: 3.781215585768223
Done training model 61 -----------------------------------------------------------------------
Now training model 62 -----------------------------------------------------------------------
Training loss: 40.517577052116394
Training loss: 27.601279735565186
Training loss: 19.226080298423767
Training loss: 14.405500411987305
Training loss: 11.265177130699158
Training loss: 9.46059113740921
Training loss: 7.820617258548737
Training loss: 6.549872234463692
Training loss: 6.257036417722702
Training los

Training loss: 42.557286500930786
Training loss: 29.433770775794983
Training loss: 20.30298513174057
Training loss: 15.313138365745544
Training loss: 12.257885873317719
Training loss: 10.314041644334793
Training loss: 9.180035531520844
Training loss: 8.071348950266838
Training loss: 6.7845723778009415
Training loss: 5.427469030022621
Training loss: 5.21216357499361
Training loss: 6.141235522925854
Training loss: 5.485828258097172
Training loss: 5.756170824170113
Training loss: 3.813890442252159
Done training model 73 -----------------------------------------------------------------------
Now training model 74 -----------------------------------------------------------------------
Training loss: 43.53428888320923
Training loss: 36.93233394622803
Training loss: 25.51536798477173
Training loss: 19.460160553455353
Training loss: 15.70228624343872
Training loss: 13.871885538101196
Training loss: 11.758798718452454
Training loss: 10.296892881393433
Training loss: 9.189627423882484
Training l

Training loss: 41.318400263786316
Training loss: 27.856123507022858
Training loss: 17.925976276397705
Training loss: 13.489124834537506
Training loss: 10.609139770269394
Training loss: 8.671275034546852
Training loss: 7.1795811876654625
Training loss: 6.26485550403595
Training loss: 7.119936063885689
Training loss: 5.743236713111401
Training loss: 5.884122282266617
Training loss: 4.719560354948044
Training loss: 5.660328924655914
Training loss: 4.86615476757288
Training loss: 3.4002290442585945
Done training model 85 -----------------------------------------------------------------------
Now training model 86 -----------------------------------------------------------------------
Training loss: 40.65844786167145
Training loss: 26.554097294807434
Training loss: 18.803741693496704
Training loss: 12.844627141952515
Training loss: 10.493435800075531
Training loss: 8.37878131866455
Training loss: 6.806018561124802
Training loss: 6.021597728133202
Training loss: 6.069346398115158
Training lo

Training loss: 43.54100036621094
Training loss: 35.172394037246704
Training loss: 21.274811267852783
Training loss: 16.042139679193497
Training loss: 12.516664832830429
Training loss: 9.984391510486603
Training loss: 7.763789668679237
Training loss: 6.204308778047562
Training loss: 7.317080199718475
Training loss: 6.849514275789261
Training loss: 5.386990070343018
Training loss: 5.585515677928925
Training loss: 4.0270708575844765
Training loss: 3.4407274797558784
Training loss: 3.862342204898596
Done training model 97 -----------------------------------------------------------------------
Now training model 98 -----------------------------------------------------------------------
Training loss: 41.86645722389221
Training loss: 27.64005994796753
Training loss: 17.015769243240356
Training loss: 12.04935896396637
Training loss: 10.146340370178223
Training loss: 5.575357332825661
Training loss: 4.597117222845554
Training loss: 4.362577028572559
Training loss: 3.7696454524993896
Training l

In [7]:
# Create loader for the test dataset
test_loader = DataLoader(testset, shuffle=True, batch_size=1, num_workers=2)

# Run each model on all of the test datasets.
preds = []
for model in all_models:
    with torch.no_grad():
        model.eval()
        model_pred = []
        for images, _ in test_loader:
            ps = torch.exp(model(images))
            top_p, top_class = ps.topk(1, dim=1)
            model_pred.append(top_class.item())
        preds.append(model_pred)


In [8]:
import numpy as np
preds = np.array(preds, dtype=int)
preds = preds.transpose(1,0)
print(preds.shape)

(10000, 100)


In [9]:
# Obtain final Labels
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=10)

    epsilon = 0.2
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

# Convert new labels into a numpy array
new_labels = np.array(new_labels, dtype=int)
new_labels.shape

(10000,)

In [16]:
## PATE Analysis
# preds = preds.transpose(1,0) # Change the shape back to number_of_teachers X number_of_examples
# print(preds.shape)

from syft.frameworks.torch.differential_privacy import pate
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=new_labels, noise_eps= 0.2, delta=1e-5)
assert data_dep_eps < data_ind_eps
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

AssertionError: 

In [15]:
preds.shape

(100, 10000)